In [43]:
from fastapi import FastAPI, Request
from fastapi.encoders import jsonable_encoder
import pickle
import os
import requests

In [44]:
app = FastAPI(debug=True)

In [45]:
filename = 'LightGBM1.pkl'
if os.path.getsize(filename) > 0:
    with open(filename, 'rb') as f:
        unpickler = pickle.Unpickler(f)
        model = unpickler.load()

In [46]:
def volume_prediction(model, vol_moving_avg, adj_close_rolling_med):
    make_prediction = model.predict([[vol_moving_avg, adj_close_rolling_med]])
    return make_prediction

In [47]:
@app.get('/')
def home():
    return {'text': 'RiskThinking WorkSample'}

In [56]:
@app.get('/predict')
def check(request: Request):
    vol_moving_avg = request.query_params['vol_moving_avg']
    adj_close_rolling_med = request.query_params['adj_close_rolling_med']
    output = volume_prediction(model, vol_moving_avg, adj_close_rolling_med)
    return jsonable_encoder({'vol_moving_avg': vol_moving_avg,
                             'adj_close_rolling_med':adj_close_rolling_med,
                             'volume': round(output[0])})
    

In [57]:
response = requests.get(url='http://127.0.0.1:8000/predict/', params={'vol_moving_avg': 12344, 'adj_close_rolling_med':20})

In [58]:
print(response)

<Response [200]>


In [59]:
response.json()

{'vol_moving_avg': 12344.0, 'adj_close_rolling_med': 20.0, 'volume': 13780}